# Gaussian

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import random

In [ ]:
mean=500
std_dev=100
n=1000
lower_bound=1
upper_bound=1000

In [ ]:
l=[]
for _ in range(n):
  num=random.gauss(mean,std_dev)
  num=max(lower_bound, min(upper_bound, num))
  l.append(num)

In [ ]:
df=pd.DataFrame(l,columns=['close'])

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
df

,close
0,388.380956
1,470.171872
2,558.675656
3,498.779420
4,652.186252
...,...
995,516.608720
996,551.013406
997,415.923692
998,379.905828


## Z-Score breakout:

If you take a sample out of this population, the sample must approximately follow the same distribution law, albeit with its sample mean and sample deviation being different from its population mean.
Hnece wwe take

In [ ]:
def generate_window_z_score(df, window, n):
  z_scores=[]

  for i in range(len(df)):
    window_data=df.iloc[i-window:i]['close']
    mean=window_data.mean()
    std_dev=window_data.std()
    z_score=(df.iloc[i]['close']-mean)/std_dev
    z_scores.append(z_score)
  df[f'zs_{n}']=z_scores
  return df

In [ ]:
df= (generate_window_z_score(df, 9, 1))
df= (generate_window_z_score(df, 16, 2))
df= (generate_window_z_score(df, 26, 3))


In [ ]:
def generate_signals(df):
  df['signal'] = 0
  a=0
  for i in range(len(df)):
    if((df['zs_1'].iloc[i]>=1.5 and df['zs_2'].iloc[i]>=1.5) or (df['zs_3'].iloc[i]>=1.5 and df['zs_1'].iloc[i]>=1.5) or (df['zs_2'].iloc[i]>=1.5 and df['zs_3'].iloc[i]>=1.5)):
      df['signal'].iloc[i]=-1
      a+=1
    if (df['zs_1'].iloc[i]<=-1.5 and df['zs_2'].iloc[i]<=-1.5) or (df['zs_3'].iloc[i]<=-1.5 and df['zs_1'].iloc[i]<=-1.5) or (df['zs_2'].iloc[i]<=-1.5 and df['zs_3'].iloc[i]<=-1.5):
      df['signal'].iloc[i]=1
      a+=1
    if (df['zs_1'].iloc[i]>=1 and df['zs_2'].iloc[i]>=1) or (df['zs_3'].iloc[i]>=1 and df['zs_1'].iloc[i]>=1) or (df['zs_2'].iloc[i]>=1 and df['zs_3'].iloc[i]>=1):
      df['signal'].iloc[i]=-1
      a+=1
    if (df['zs_1'].iloc[i]<=-1.5 and df['zs_2'].iloc[i]<=-1) or (df['zs_3'].iloc[i]<=-1 and df['zs_1'].iloc[i]<=-1) or (df['zs_2'].iloc[i]<=-1 and df['zs_3'].iloc[i]<=-1):
      df['signal'].iloc[i]=1
      a+=1
  return df

In [ ]:
df = generate_signals(df)

## We can also use confidence level values to change our capital dynamically, which is given by
\[
\F(z) = \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{z} e^{-\frac{t^2}{2}} \, dt
\]
## Where F(z) represents the cumulative distribution function, hence confidence of going down is F(z) and that of going up is 1-F(z)


In [ ]:
df['Close']=df['close']
df['signals']=df['signal']

In [ ]:
df

,close,zs_1,zs_2,zs_3,signal,Close,signals
0,388.380956,NaN,NaN,NaN,0,388.380956,0
1,470.171872,NaN,NaN,NaN,0,470.171872,0
2,558.675656,NaN,NaN,NaN,0,558.675656,0
3,498.779420,NaN,NaN,NaN,0,498.779420,0
4,652.186252,NaN,NaN,NaN,0,652.186252,0
...,...,...,...,...,...,...,...
995,516.608720,0.300527,0.271499,0.516881,0,516.608720,0
996,551.013406,0.613910,0.606033,0.857101,0,551.013406,0
997,415.923692,-0.690425,-0.876894,-0.762253,0,415.923692,0
998,379.905828,-1.005100,-1.220732,-1.157192,1,379.905828,1


Dynamic confidence level based capital allocation

In [ ]:
from scipy import stats

def generate_confidence_level(df):
    df['confidence']=0
    df['rolling mean']=df['Close'].rolling(window=10).mean()
    df['rolling std']=df['Close'].rolling(window=10).std()
    for i in range(len(df)):
        if pd.notna(df['rolling mean'].iloc[i]) and pd.notna(df['rolling std'].iloc[i]):
            if df['signals'].iloc[i]>0:
                df.at[i,'confidence']=stats.norm.cdf((df['Close'].iloc[i]-df['rolling mean'].iloc[i])/df['rolling std'].iloc[i])
            if df['signals'].iloc[i]<0:
                df.at[i,'confidence']=1-stats.norm.cdf((df['Close'].iloc[i]-df['rolling mean'].iloc[i])/df['rolling std'].iloc[i])
    return df

df=generate_confidence_level(df)
df


,close,zs_1,zs_2,zs_3,signal,Close,signals,confidence,rolling mean,rolling std
0,388.380956,NaN,NaN,NaN,0,388.380956,0,0.000000,NaN,NaN
1,470.171872,NaN,NaN,NaN,0,470.171872,0,0.000000,NaN,NaN
2,558.675656,NaN,NaN,NaN,0,558.675656,0,0.000000,NaN,NaN
3,498.779420,NaN,NaN,NaN,0,498.779420,0,0.000000,NaN,NaN
4,652.186252,NaN,NaN,NaN,0,652.186252,0,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
995,516.608720,0.300527,0.271499,0.516881,0,516.608720,0,0.000000,488.268197,99.288819
996,551.013406,0.613910,0.606033,0.857101,0,551.013406,0,0.000000,492.922448,101.205093
997,415.923692,-0.690425,-0.876894,-0.762253,0,415.923692,0,0.000000,482.275474,103.338133
998,379.905828,-1.005100,-1.220732,-1.157192,1,379.905828,1,0.181625,477.395996,107.227750


In [ ]:
print(df['confidence'].min())
print(df['confidence'].max())
print(df['confidence'].mean())

0.0
0.2621209004431898
0.03336694556015115


In [ ]:
print(df['signals'].value_counts())

signals
 0    665
-1    178
 1    157
Name: count, dtype: int64


# Backtester inshallah

In [ ]:
def iron_out(df):
  flag=0
  for i in range(len(df)):
    if flag==0 and df['signals'].iloc[i]!=0:
      flag=df['signals'].iloc[i]
    if flag==1 and df['signals'].iloc[i]>0:
      df['signals'].iloc[i]=0
    if flag==-1 and df['signals'].iloc[i]<0:
      df['signals'].iloc[i]=0
    if flag==-1 and df['signals'].iloc[i]>0:
      flag=0
    if flag==1 and df['signals'].iloc[i]<0:
      flag=0
  return df

In [ ]:
df=iron_out(df)
print(df['signals'].value_counts())

signals
 0    883
 1     61
-1     56
Name: count, dtype: int64


In [ ]:
df['signal']=df['signals']

In [ ]:
import numpy as np

def backtesting_engine(df):
 capital=100000
 portfolio=[]
 position=0
 winning_trades=0
 losing_trades=0
 returns=[]
 stocks_in_hand=0
 stop_loss=0.05
 stop_loss_price=0
 start_price=0

 for i in range(len(df)):
  if position==0 and df['signal'].iloc[i]==1:
   position=1
   used_capital=0.5*(1+df['confidence'].iloc[i])*capital
   start_price=df['Close'].iloc[i]
   stocks_in_hand=used_capital/df['Close'].iloc[i]
   capital-=used_capital
   stop_loss_price=start_price*(1-stop_loss*df['confidence'].iloc[i])
  elif position==0 and df['signal'].iloc[i]==-1:
   position=-1
   used_capital=0.5*(1+df['confidence'].iloc[i])*capital
   start_price=df['Close'].iloc[i]
   stocks_in_hand=used_capital/df['Close'].iloc[i]
   capital+=used_capital
   stop_loss_price=start_price*(1+stop_loss*df['confidence'].iloc[i])
  elif position==1:
   if df['signal'].iloc[i]==-1 or df['Close'].iloc[i]<stop_loss_price:
    current_return=((df['Close'].iloc[i]-start_price)/start_price)*100
    returns.append(current_return)
    capital+=stocks_in_hand*df['Close'].iloc[i]
    stocks_in_hand=0
    position=0
    if current_return>0:
     winning_trades+=1
    else:
     losing_trades+=1
  elif position==-1:
   if df['signal'].iloc[i]==1 or df['Close'].iloc[i]>stop_loss_price:
    current_return=((start_price-df['Close'].iloc[i])/start_price)*100
    returns.append(current_return)
    capital-=stocks_in_hand*df['Close'].iloc[i]
    stocks_in_hand=0
    position=0
    if current_return>0:
     winning_trades+=1
    else:
     losing_trades+=1

  if position==1:
   portfolio_today=stocks_in_hand*df['Close'].iloc[i]+capital
  elif position==-1:
   portfolio_today=capital-stocks_in_hand*df['Close'].iloc[i]
  else:
   portfolio_today=capital
  portfolio.append(portfolio_today)

 if position==1:
  portfolio_today=stocks_in_hand*df['Close'].iloc[-1]+capital
 elif position==-1:
  portfolio_today=capital-stocks_in_hand*df['Close'].iloc[-1]
 portfolio.append(portfolio_today)

 total_returns=(portfolio_today-100000)/100000
 if df['Close'].iloc[0]!=0:
  benchmark_returns=((df['Close'].iloc[-1]-df['Close'].iloc[0])*(100000/df['Close'].iloc[0]))/100000
  benchmark_returns=(benchmark_returns-3.73)
 else:
  benchmark_returns=float('nan')
 std_returns=np.std(returns) if len(returns)>0 else float('nan')
 sharpe_ratio=(total_returns-3.73)/std_returns if std_returns!=0 else float('nan')

 maximum_drawdown=0
 peak=100000
 for value in portfolio:
  if value>peak:
   peak=value
  else:
   drawdown=(value-peak)/peak*100
   if drawdown<maximum_drawdown:
    maximum_drawdown=drawdown

 print(f"Total Returns: {total_returns}%")
 print(f"Benchmark Returns: {benchmark_returns}%")
 print(f"Sharpe Ratio: {sharpe_ratio}")
 print(f"Winning Trades: {winning_trades}")
 print(f"Losing Trades: {losing_trades}")
 print(f"Maximum Drawdown: {maximum_drawdown}%")

 return total_returns,benchmark_returns,sharpe_ratio,maximum_drawdown,winning_trades,losing_trades


In [ ]:
df1=df.copy()

In [ ]:
backtesting_engine(df)

Total Returns: 958337688.8164748%
Benchmark Returns: -3.4887823803461213%
Sharpe Ratio: 21566744.93661683
Winning Trades: 87
Losing Trades: 56
Maximum Drawdown: -44.557982958839844%


(958337688.8164748,
 -3.4887823803461213,
 21566744.93661683,
 -44.557982958839844,
 87,
 56)



------







# Exponential

In [ ]:
lambda_param=0.002
n=1000
low=400
upper=1000

In [ ]:
random_numbers=np.random.exponential(scale=1/lambda_param,size=n)
random.numbers= np.clip(random_numbers,low,upper)
l=random_numbers.tolist()

In [ ]:
data=pd.DataFrame(l,columns=['close'])

In [ ]:
data['Close']=data['close']

In [ ]:
data

,close,Close
0,285.977376,285.977376
1,42.305140,42.305140
2,145.172261,145.172261
3,906.911524,906.911524
4,473.426994,473.426994
...,...,...
995,1821.578853,1821.578853
996,90.711058,90.711058
997,500.114518,500.114518
998,462.741454,462.741454


# First a Markov Model was used to find out the probability of price transitioning from one bin to the tother. Bins simply represent an interval in which the closing price may lie.
No this implementation is particularly useful for this PS because the prices are randomized, in real world prices wont just fall or rise by 1000% but here they can, so the transition matrix wont be a nearly diagonal matrix with values sharply declining as you go away from the diagonal (as it happens in most continous price data), rather it will now store the conditional probability of the price going up or down
Two columns were created, up_prob and down_prob, by summing the conditional probabilites of higher and lower bins, a confidence level was also determined which was used to alter the stop loss, take profit, and the capital allocation,

In [ ]:
class MarkovModel:
    def __init__(self,df,num_states):
        self.df=df
        self.num_states=num_states
        self.bins=None
        self.markov_matrix=None

    def create_bins(self):
        self.bins=np.linspace(self.df['Close'].min(),self.df['Close'].max(),self.num_states+1)
        self.df['State']=np.digitize(self.df['Close'],self.bins)-1
        self.df['current_bin']=np.roll(self.df['State'],1)
        return self.df,self.bins

    def transition_matrix(self):
        self.markov_matrix=np.zeros((self.num_states,self.num_states))
        for i in range(len(self.df)-1):
            current_state=self.df['State'].iloc[i]
            next_state=self.df['State'].iloc[i+1]
            self.markov_matrix[current_state-1][next_state-1]+=1
        row_sums=self.markov_matrix.sum(axis=1,keepdims=True)
        row_sums[row_sums==0]=1
        self.markov_matrix/=row_sums
        return self.markov_matrix

    def calculate_net_probabilities(self):
        self.df['up_prob']=0
        self.df['down_prob']=0
        for i in range(len(self.df)):
            current_state=self.df['State'].iloc[i]
            up_prob=np.sum(self.markov_matrix[current_state-1,:][current_state-1:])
            down_prob=np.sum(self.markov_matrix[current_state-1,:current_state-1])
            self.df['up_prob'].iloc[i]=up_prob
            self.df['down_prob'].iloc[i]=down_prob
        return self.df


In [ ]:
num_states=40
markov_model=MarkovModel(data,num_states)
markov_model.create_bins()
markov_model.transition_matrix()
data_with_probs=markov_model.calculate_net_probabilities()

print(data_with_probs[['Close','up_prob','down_prob']])


           Close   up_prob  down_prob
0     285.977376  0.899225   0.100775
1      42.305140  0.170984   0.829016
2     145.172261  1.000000   0.000000
3     906.911524  0.432432   0.567568
4     473.426994  0.643836   0.356164
..           ...       ...        ...
995  1821.578853  0.400000   0.600000
996    90.711058  0.170984   0.829016
997   500.114518  0.643836   0.356164
998   462.741454  0.643836   0.356164
999   249.088109  0.899225   0.100775

[1000 rows x 3 columns]


In [ ]:
print(data_with_probs['up_prob'].min())
print(data_with_probs['up_prob'].max())
print(data_with_probs['up_prob'].mean())
print(data_with_probs['down_prob'].min())
print(data_with_probs['down_prob'].max())
print(data_with_probs['down_prob'].mean())

0.0
1.0
0.5728992248062016
0.0
1.0
0.4271007751937984


In [ ]:
data_with_probs

,close,Close,State,current_bin,up_prob,down_prob
0,285.977376,285.977376,2,2,0.899225,0.100775
1,42.305140,42.305140,0,2,0.170984,0.829016
2,145.172261,145.172261,1,0,1.000000,0.000000
3,906.911524,906.911524,8,1,0.432432,0.567568
4,473.426994,473.426994,4,8,0.643836,0.356164
...,...,...,...,...,...,...
995,1821.578853,1821.578853,17,11,0.400000,0.600000
996,90.711058,90.711058,0,17,0.170984,0.829016
997,500.114518,500.114518,4,0,0.643836,0.356164
998,462.741454,462.741454,4,4,0.643836,0.356164


In [ ]:
def generate_mah_signals(df):
  df['signal']=0
  for i in range(len(df)):
    if df['up_prob'].iloc[i]>=0.7 and df['down_prob'].iloc[i]<0.3:
      df['signal'].iloc[i]=1
    if df['up_prob'].iloc[i]<0.3 and df['down_prob'].iloc[i]>=0.7:
      df['signal'].iloc[i]=-1
  return df
d1= data_with_probs.copy()
d1=generate_mah_signals(d1)
print(d1['signal'].value_counts())

signal
 1    382
 0    342
-1    276
Name: count, dtype: int64


In [ ]:
def confidence(df):
  df['confidence']=0
  for i in range(len(df)):
   if df['signal'].iloc[i]>0:
      df['confidence'].iloc[i]= df['up_prob'].iloc[i]-0.7
   if df['signal'].iloc[i]<0:
      df['confidence'].iloc[i]= df['down_prob'].iloc[i]-0.7
  return df
d1=confidence(d1)
d1


,close,Close,State,current_bin,up_prob,down_prob,signal,confidence
0,285.977376,285.977376,2,2,0.899225,0.100775,1,0.199225
1,42.305140,42.305140,0,2,0.170984,0.829016,-1,0.129016
2,145.172261,145.172261,1,0,1.000000,0.000000,1,0.300000
3,906.911524,906.911524,8,1,0.432432,0.567568,0,0.000000
4,473.426994,473.426994,4,8,0.643836,0.356164,0,0.000000
...,...,...,...,...,...,...,...,...
995,1821.578853,1821.578853,17,11,0.400000,0.600000,0,0.000000
996,90.711058,90.711058,0,17,0.170984,0.829016,-1,0.129016
997,500.114518,500.114518,4,0,0.643836,0.356164,0,0.000000
998,462.741454,462.741454,4,4,0.643836,0.356164,0,0.000000


In [ ]:
d1['signals']=d1['signal']

In [ ]:
backtesting_engine(d1)

Total Returns: 4.071396707880698e+50%
Benchmark Returns: -3.8589936543606984%
Sharpe Ratio: 2.6538644905701272e+47
Winning Trades: 67
Losing Trades: 214
Maximum Drawdown: -72713.26041509015%


(4.071396707880698e+50,
 -3.8589936543606984,
 2.6538644905701272e+47,
 -72713.26041509015,
 67,
 214)